In [30]:
import pandas as pd
import numpy as np
#from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler



In [2]:
rawDataDf = pd.read_csv("Raw_Data_Set.csv")

rawDataDf.head()

,PITCHNO,TM_DATE,TM_TIME,UTCDATE,UTCTIME,LOCALDATETIME,UTCDATETIME,PAOFINNING,PITCHOFPA,PITCHER,...,LASTTRACKEDDISTANCE,BEARING,HANGTIME,PFXX,PFXZ,HOMETEAM,AWAYTEAM,STADIUM,LEVEL,GAMEID
0,1,2021-04-20,17:23.9,00:00.0,17:23.9,2021-04-20 19:17:23.8830000 -07:00,2021-04-21 02:17:23.8830000 +00:00,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,SD,MIL,Petco,MLB,20210420-Petco-1
1,2,2021-04-20,17:24.9,00:00.0,17:24.9,2021-04-20 19:17:24.8830000 -07:00,2021-04-21 02:17:24.8830000 +00:00,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,SD,MIL,Petco,MLB,20210420-Petco-1
2,3,2021-04-20,17:25.9,00:00.0,17:25.9,2021-04-20 19:17:25.8830000 -07:00,2021-04-21 02:17:25.8830000 +00:00,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,SD,MIL,Petco,MLB,20210420-Petco-1
3,4,2021-04-20,17:26.9,00:00.0,17:26.9,2021-04-20 19:17:26.8830000 -07:00,2021-04-21 02:17:26.8830000 +00:00,2,3,NaN,...,NaN,NaN,NaN,NaN,NaN,SD,MIL,Petco,MLB,20210420-Petco-1
4,5,2021-04-20,17:27.9,00:00.0,17:27.9,2021-04-20 19:17:27.8830000 -07:00,2021-04-21 02:17:27.8830000 +00:00,2,4,NaN,...,NaN,NaN,NaN,NaN,NaN,SD,MIL,Petco,MLB,20210420-Petco-1


In [3]:
win_loss_DF = pd.read_csv("win-loss percentage.csv")
win_loss_DF.head()

,Rank,Team,Win-Loss Percentage
0,1,LA,0.685
1,2,HOU,0.654
2,3,Atlanta Braves,0.623
3,4,New York Mets,0.623
4,5,NYY,0.611


In [4]:
list(rawDataDf.columns)


['PITCHNO',
 'TM_DATE',
 'TM_TIME',
 'UTCDATE',
 'UTCTIME',
 'LOCALDATETIME',
 'UTCDATETIME',
 'PAOFINNING',
 'PITCHOFPA',
 'PITCHER',
 'PITCHERID',
 'PITCHERTHROWS',
 'PITCHERTEAM',
 'BATTER',
 'BATTERID',
 'BATTERSIDE',
 'BATTERTEAM',
 'PITCHERSET',
 'INNING',
 'TOP_BOTTOM',
 'OUTS',
 'BALLS',
 'STRIKES',
 'PITCH_TYPE',
 'PITCHCALL',
 'KORBB',
 'TAGGEDHITTYPE',
 'AUTOHITTYPE',
 'PLAYRESULT',
 'OUTSONPLAY',
 'RUNSSCORED',
 'NOTES',
 'RELSPEED',
 'VERTRELANGLE',
 'HORZRELANGLE',
 'SPINRATE',
 'SPINAXIS',
 'TILT',
 'RELHEIGHT',
 'RELSIDE',
 'EXTENSION',
 'VERTBREAK',
 'INDUCEDVERTBREAK',
 'HORZBREAK',
 'PLATELOCHEIGHT',
 'PLATELOCSIDE',
 'ZONESPEED',
 'VERTAPPRANGLE',
 'HORZAPPRANGLE',
 'ZONETIME',
 'EXITSPEED',
 'ANGLE',
 'DIRECTION',
 'HITSPINRATE',
 'POSITIONAT110X',
 'POSITIONAT110Y',
 'POSITIONAT110Z',
 'DISTANCE',
 'LASTTRACKEDDISTANCE',
 'BEARING',
 'HANGTIME',
 'PFXX',
 'PFXZ',
 'HOMETEAM',
 'AWAYTEAM',
 'STADIUM',
 'LEVEL',
 'GAMEID']

In [5]:
rawDataDf.PLAYRESULT.value_counts()

Undefined         12136
Out                1559
Single              518
Double              173
HomeRun             104
Error                42
FieldersChoice       40
Sacrifice            22
Triple               13
Name: PLAYRESULT, dtype: int64

In [6]:
#strikeouts and walks didn't have a play result

rawDataDf.loc[rawDataDf['KORBB'] == 'Strikeout', 'PLAYRESULT'] = 'Out'
rawDataDf.loc[rawDataDf['KORBB'] == 'Strikeout', 'OUTSONPLAY'] = 1



rawDataDf.loc[rawDataDf['KORBB'] == 'Walk', 'PLAYRESULT'] = 'Walk'

rawDataDf['PLAYRESULT'].head(20)

0           Out
1     Undefined
2     Undefined
3     Undefined
4           Out
5     Undefined
6     Undefined
7     Undefined
8     Undefined
9           Out
10    Undefined
11    Undefined
12    Undefined
13    Undefined
14          Out
15    Undefined
16    Undefined
17    Undefined
18    Undefined
19    Undefined
Name: PLAYRESULT, dtype: object

In [7]:
#see all results for pitchcall
rawDataDf.PITCHCALL.value_counts()

BallCalled         4955
StrikeCalled       2578
FoulBall           2476
InPlay             2471
StrikeSwinging     1840
BallinDirt          198
HitByPitch           50
BallIntentional      39
Name: PITCHCALL, dtype: int64

In [8]:
rawDataDf.to_csv("checking_it.csv")

In [9]:
#create new dataframe with desired info for ML - including pitchers and hitters
desiredDataDF = rawDataDf[['PITCHERTEAM', 'BATTERTEAM', 'PITCH_TYPE','PITCHCALL', 'PLAYRESULT', 'OUTSONPLAY', 'RUNSSCORED',
                          'AUTOHITTYPE','RELSPEED', 'SPINRATE', 'PLATELOCSIDE', 'PLATELOCHEIGHT']].copy()

desiredDataDF.head(20)

,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,PLATELOCSIDE,PLATELOCHEIGHT
0,SD,MIL,NaN,InPlay,Out,1,0,NaN,NaN,NaN,NaN,NaN
1,SD,MIL,NaN,StrikeCalled,Undefined,0,0,NaN,NaN,NaN,NaN,NaN
2,SD,MIL,NaN,StrikeCalled,Undefined,0,0,NaN,NaN,NaN,NaN,NaN
3,SD,MIL,NaN,FoulBall,Undefined,0,0,NaN,NaN,NaN,NaN,NaN
4,SD,MIL,NaN,StrikeSwinging,Out,1,0,NaN,NaN,NaN,NaN,NaN
5,SD,MIL,NaN,FoulBall,Undefined,0,0,NaN,NaN,NaN,NaN,NaN
6,SD,MIL,NaN,BallCalled,Undefined,0,0,NaN,NaN,NaN,NaN,NaN
7,SD,MIL,NaN,FoulBall,Undefined,0,0,NaN,NaN,NaN,NaN,NaN
8,SD,MIL,NaN,BallCalled,Undefined,0,0,NaN,NaN,NaN,NaN,NaN
9,SD,MIL,NaN,StrikeSwinging,Out,1,0,NaN,NaN,NaN,NaN,NaN


In [10]:
#add win percentage for pitching team
desiredDataDF = pd.merge(desiredDataDF, win_loss_DF, left_on="PITCHERTEAM", right_on="Team", how="outer")
desiredDataDF.head()


,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,PLATELOCSIDE,PLATELOCHEIGHT,Rank,Team,Win-Loss Percentage
0,SD,MIL,NaN,InPlay,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549
1,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549
2,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549
3,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549
4,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549


In [11]:
#rename win loss for specifically pitcher team

desiredDataDF = desiredDataDF.rename(columns={"Win-Loss Percentage": "Pitcher: WL Percentage", "Rank": "Pitcher Rank", 
                                      "Team": "Pitcher Team"})
desiredDataDF.head()

,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,PLATELOCSIDE,PLATELOCHEIGHT,Pitcher Rank,Pitcher Team,Pitcher: WL Percentage
0,SD,MIL,NaN,InPlay,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549
1,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549
2,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549
3,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549
4,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549


In [12]:
#add win percentage for batting team
desiredDataDF = pd.merge(desiredDataDF, win_loss_DF, left_on="BATTERTEAM", right_on="Team", how="inner")
desiredDataDF.head()

,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,PLATELOCSIDE,PLATELOCHEIGHT,Pitcher Rank,Pitcher Team,Pitcher: WL Percentage,Rank,Team,Win-Loss Percentage
0,SD,MIL,NaN,InPlay,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
1,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
2,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
3,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
4,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531


In [13]:
#rename columns
desiredDataDF = desiredDataDF.rename(columns={"Win-Loss Percentage": "Batter: WL Percentage", "Rank": "Batter Rank", 
                                      "Team": "Batter Team"})
desiredDataDF.head()

,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,PLATELOCSIDE,PLATELOCHEIGHT,Pitcher Rank,Pitcher Team,Pitcher: WL Percentage,Batter Rank,Batter Team,Batter: WL Percentage
0,SD,MIL,NaN,InPlay,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
1,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
2,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
3,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
4,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531


In [14]:
#expore to csv for visual story creation
desiredDataDF.to_csv("desiredData.csv")

desiredDataDF.to_excel("desiredData.xlsx")


In [15]:
#create new df to convert categorical to quantitative
ML_dataDF = desiredDataDF

ML_dataDF.head()

,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,PLATELOCSIDE,PLATELOCHEIGHT,Pitcher Rank,Pitcher Team,Pitcher: WL Percentage,Batter Rank,Batter Team,Batter: WL Percentage
0,SD,MIL,NaN,InPlay,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
1,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
2,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
3,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531
4,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531


In [16]:
#pitchcall ball or strike

ML_dataDF.loc[ML_dataDF['PITCHCALL'] == 'BallCalled', 'Ball/Strike'] = 'Ball'
ML_dataDF.loc[ML_dataDF['PITCHCALL'] == 'BallinDirt', 'Ball/Strike'] = 'Ball'
ML_dataDF.loc[ML_dataDF['PITCHCALL'] == 'BallIntentional', 'Ball/Strike'] = 'Ball'
ML_dataDF.loc[ML_dataDF['PITCHCALL'] == 'HitByPitch', 'Ball/Strike'] = 'Ball'

ML_dataDF.loc[ML_dataDF['PITCHCALL'] == 'StrikeCalled', 'Ball/Strike'] = 'Strike'
ML_dataDF.loc[ML_dataDF['PITCHCALL'] == 'FoulBall', 'Ball/Strike'] = 'Strike'
ML_dataDF.loc[ML_dataDF['PITCHCALL'] == 'StrikeSwinging', 'Ball/Strike'] = 'Strike'
ML_dataDF.loc[ML_dataDF['PITCHCALL'] == 'InPlay', 'Ball/Strike'] = 'Strike'

ML_dataDF.loc[ML_dataDF['Ball/Strike'] == 'Ball', 'Num_Ball/Strike'] = 0
ML_dataDF.loc[ML_dataDF['Ball/Strike'] == 'Strike', 'Num_Ball/Strike'] = 1

ML_dataDF.head(20)

,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,PLATELOCSIDE,PLATELOCHEIGHT,Pitcher Rank,Pitcher Team,Pitcher: WL Percentage,Batter Rank,Batter Team,Batter: WL Percentage,Ball/Strike,Num_Ball/Strike
0,SD,MIL,NaN,InPlay,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0
1,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0
2,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0
3,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0
4,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0
5,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0
6,SD,MIL,NaN,BallCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Ball,0.0
7,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0
8,SD,MIL,NaN,BallCalled,Undefined,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Ball,0.0
9,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,NaN,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0


In [17]:
#playresult to numeric
#Undefined        0 
#Out             -1 
#Single           1    
#Double           2   
#HomeRun          4   
#Error            1   
#FieldersChoice  -1     
#Sacrifice       -1   
#Triple           3
#Walk             1

ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'Undefined', 'ResultNum'] = 0
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'Out', 'ResultNum'] = -1
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'Single', 'ResultNum'] = 1
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'Double', 'ResultNum'] = 2
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'HomeRun', 'ResultNum'] = 4
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'Error', 'ResultNum'] = 1
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'FieldersChoice', 'ResultNum'] = -1
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'Sacrifice', 'ResultNum'] = -1
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'Triple', 'ResultNum'] = 3
ML_dataDF.loc[ML_dataDF['PLAYRESULT'] == 'Walk', 'ResultNum'] = 1

ML_dataDF.head()

,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,...,PLATELOCHEIGHT,Pitcher Rank,Pitcher Team,Pitcher: WL Percentage,Batter Rank,Batter Team,Batter: WL Percentage,Ball/Strike,Num_Ball/Strike,ResultNum
0,SD,MIL,NaN,InPlay,Out,1.0,0.0,NaN,NaN,NaN,...,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0,-1.0
1,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,...,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0,0.0
2,SD,MIL,NaN,StrikeCalled,Undefined,0.0,0.0,NaN,NaN,NaN,...,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0,0.0
3,SD,MIL,NaN,FoulBall,Undefined,0.0,0.0,NaN,NaN,NaN,...,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0,0.0
4,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,...,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0,-1.0


In [18]:
ML_dataDF.dtypes

PITCHERTEAM                object
BATTERTEAM                 object
PITCH_TYPE                 object
PITCHCALL                  object
PLAYRESULT                 object
OUTSONPLAY                float64
RUNSSCORED                float64
AUTOHITTYPE                object
RELSPEED                  float64
SPINRATE                  float64
PLATELOCSIDE              float64
PLATELOCHEIGHT            float64
Pitcher Rank                int64
Pitcher Team               object
Pitcher: WL Percentage    float64
Batter Rank                 int64
Batter Team                object
Batter: WL Percentage     float64
Ball/Strike                object
Num_Ball/Strike           float64
ResultNum                 float64
dtype: object

In [19]:
#DF without undefined playresult
ML_dataDF_plays = ML_dataDF[ML_dataDF.PLAYRESULT != 'Undefined']

ML_dataDF_plays.head()

,PITCHERTEAM,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,...,PLATELOCHEIGHT,Pitcher Rank,Pitcher Team,Pitcher: WL Percentage,Batter Rank,Batter Team,Batter: WL Percentage,Ball/Strike,Num_Ball/Strike,ResultNum
0,SD,MIL,NaN,InPlay,Out,1.0,0.0,NaN,NaN,NaN,...,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0,-1.0
4,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,...,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0,-1.0
9,SD,MIL,NaN,StrikeSwinging,Out,1.0,0.0,NaN,NaN,NaN,...,NaN,10,SD,0.549,12,MIL,0.531,Strike,1.0,-1.0
13,SD,MIL,CH,InPlay,Out,1.0,0.0,FlyBall,85.97727,1726.874507,...,1.54898,10,SD,0.549,12,MIL,0.531,Strike,1.0,-1.0
21,SD,MIL,FA,InPlay,Out,1.0,0.0,LineDrive,96.65576,2270.553169,...,2.40620,10,SD,0.549,12,MIL,0.531,Strike,1.0,-1.0


In [20]:
ML_dataDF_plays.to_excel("desiredData_plays.xlsx")


In [21]:
#df groupby pitching team
df_groupby_pitching = ML_dataDF.groupby(['PITCHERTEAM'])

df_groupby_pitching.dtypes

,BATTERTEAM,PITCH_TYPE,PITCHCALL,PLAYRESULT,OUTSONPLAY,RUNSSCORED,AUTOHITTYPE,RELSPEED,SPINRATE,PLATELOCSIDE,PLATELOCHEIGHT,Pitcher Rank,Pitcher Team,Pitcher: WL Percentage,Batter Rank,Batter Team,Batter: WL Percentage,Ball/Strike,Num_Ball/Strike,ResultNum
PITCHERTEAM,,,,,,,,,,,,,,,,,,,,
ANA,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64
ARI,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64
BAL,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64
BOS,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64
CIN,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64
COL,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64
CWS,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64
HOU,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64
KC,object,object,object,object,float64,float64,object,float64,float64,float64,float64,int64,object,float64,int64,object,float64,object,float64,float64


In [22]:
#df groupby hitting team
df_groupby_hitting = ML_dataDF.groupby(['BATTERTEAM'])

## Machine Learning By Pitching Team

In [23]:
#ML DF pitching team with undefined
Pitch_ML_DF = ML_dataDF[['OUTSONPLAY', 'RUNSSCORED', 'Num_Ball/Strike','ResultNum', 
                          'Pitcher: WL Percentage']].copy()

Pitch_ML_DF.head()



,OUTSONPLAY,RUNSSCORED,Num_Ball/Strike,ResultNum,Pitcher: WL Percentage
0,1.0,0.0,1.0,-1.0,0.549
1,0.0,0.0,1.0,0.0,0.549
2,0.0,0.0,1.0,0.0,0.549
3,0.0,0.0,1.0,0.0,0.549
4,1.0,0.0,1.0,-1.0,0.549


In [24]:
#ML DF batter team with undefined
Bat_ML_DF = ML_dataDF[['OUTSONPLAY', 'RUNSSCORED', 'Num_Ball/Strike','ResultNum', 
                          'Batter: WL Percentage']].copy()

Bat_ML_DF.head()

,OUTSONPLAY,RUNSSCORED,Num_Ball/Strike,ResultNum,Batter: WL Percentage
0,1.0,0.0,1.0,-1.0,0.531
1,0.0,0.0,1.0,0.0,0.531
2,0.0,0.0,1.0,0.0,0.531
3,0.0,0.0,1.0,0.0,0.531
4,1.0,0.0,1.0,-1.0,0.531


In [25]:
#ML DF pitching team omitting undefined
Pitch_Plays_ML_DF = ML_dataDF_plays[['OUTSONPLAY', 'RUNSSCORED', 'Num_Ball/Strike','ResultNum', 
                          'Pitcher: WL Percentage']].copy()

Pitch_ML_DF.head()


,OUTSONPLAY,RUNSSCORED,Num_Ball/Strike,ResultNum,Pitcher: WL Percentage
0,1.0,0.0,1.0,-1.0,0.549
1,0.0,0.0,1.0,0.0,0.549
2,0.0,0.0,1.0,0.0,0.549
3,0.0,0.0,1.0,0.0,0.549
4,1.0,0.0,1.0,-1.0,0.549


In [26]:
#ML with undefined (pitching team)
X = Pitch_ML_DF[['OUTSONPLAY', 'RUNSSCORED', 'Num_Ball/Strike', 'ResultNum']]
y = Pitch_ML_DF[['Pitcher: WL Percentage']]

In [27]:
print(X)

       OUTSONPLAY  RUNSSCORED  Num_Ball/Strike  ResultNum
0             1.0         0.0              1.0       -1.0
1             0.0         0.0              1.0        0.0
2             0.0         0.0              1.0        0.0
3             0.0         0.0              1.0        0.0
4             1.0         0.0              1.0       -1.0
...           ...         ...              ...        ...
14602         0.0         0.0              0.0        0.0
14603         0.0         0.0              0.0        0.0
14604         0.0         0.0              0.0        0.0
14605         0.0         0.0              1.0        0.0
14606         1.0         0.0              1.0       -1.0

[14607 rows x 4 columns]


# Linear Regression for Pitching Team, all pitches

In [31]:
# create MinMaxScaler object
scaler = MinMaxScaler()

In [34]:
# fit and transform the data
X_scaled = scaler.fit_transform(X)

In [42]:
#create lin reg ML
pitch_model = LinearRegression()

In [53]:
pitch_model.fit(X_scaled, y)

LinearRegression()

In [54]:
print("Coefficients: ", pitch_model.coef_)
print("Intercept: ", pitch_model.intercept_)

Coefficients:  [[-0.00452024 -0.0063252   0.00122013 -0.00821774]]
Intercept:  [0.50406662]


In [63]:
#calculate and print r-value
r_squared = pitch_model.score(X_scaled, y)
print("Correlation between Pitcher and Record (all pitches):", r_squared)

Correlation between Pitcher and Record (all pitches): 0.0002769175762997733


## Linear Regression for Pitching Team, only plays

In [48]:
#ML without undefined (pitching team) (only plays)
X_pitch_plays = Pitch_Plays_ML_DF[['OUTSONPLAY', 'RUNSSCORED', 'Num_Ball/Strike', 'ResultNum']]
y_pitch_plays = Pitch_Plays_ML_DF[['Pitcher: WL Percentage']]

In [56]:
#fit and transform the data
X_pitch_plays_scaled = scaler.fit_transform(X_pitch_plays)

In [57]:
#create lin reg ML for plays
pitch_model_plays = LinearRegression()

In [58]:
pitch_model_plays.fit(X_pitch_plays_scaled, y_pitch_plays)

LinearRegression()

In [62]:
#calculate and print r-value
r_squared_pitch_plays = pitch_model_plays.score(X_pitch_plays_scaled, y_pitch_plays)
print("Correlation between Pitcher and Record (only plays):", r_squared_pitch_plays)

Correlation between Pitcher and Record (only plays): 0.0007343224227988054


## Linear Regression for Batter Team, all pitches

In [65]:
#define variables for lin reg
X_bat = Bat_ML_DF[['OUTSONPLAY', 'RUNSSCORED', 'Num_Ball/Strike', 'ResultNum']]
y_bat = Bat_ML_DF[['Batter: WL Percentage']]

In [66]:
#fit and transform the data
X_bat_scaled = scaler.fit_transform(X_bat)

In [69]:
#create lin reg for batters
bat_model = LinearRegression()

In [70]:
#lin reg
bat_model.fit(X_bat_scaled, y_bat)

LinearRegression()

In [71]:
#calculate and print r-value
r_squared_bat = bat_model.score(X_bat_scaled, y_bat)
print("Correlation between Batter and Record:", r_squared_bat)

Correlation between Batter and Record: 0.0002116597581568458


## Linear Regression for Batter Team, only plays

In [72]:
#new df for batters, plays only
Bat_Plays_ML_DF = ML_dataDF_plays[['OUTSONPLAY', 'RUNSSCORED', 'Num_Ball/Strike','ResultNum', 
                               'Batter: WL Percentage']].copy()

In [73]:
#define variables for lin reg
X_bat_plays = Bat_Plays_ML_DF[['OUTSONPLAY', 'RUNSSCORED', 'Num_Ball/Strike', 'ResultNum']]
y_bat_plays = Bat_Plays_ML_DF[['Batter: WL Percentage']]

In [74]:
#fit and transform the data
X_bat_plays_scaled = scaler.fit_transform(X_bat_plays)

In [75]:
#create lin reg for batters
bat_plays_model = LinearRegression()

In [76]:
#lin reg
bat_plays_model.fit(X_bat_plays_scaled, y_bat_plays)

LinearRegression()

In [77]:
#calculate and print r-value
r_squared_bat_plays = bat_plays_model.score(X_bat_plays_scaled, y_bat_plays)
print("Correlation between Batter and Record (only plays):", r_squared_bat_plays)

Correlation between Batter and Record (only plays): 0.0015267205914351045
